reference: https://www.quantopian.com/tutorials/getting-started#lesson6

### Data Processing in Algorithms

The next step will be to integrate the data pipeline we built in Research into our algorithm. One important distinction from Research is that during a backtest our pipeline will be executed each day as the simulation progresses, so we won't need to include start_date and end_date arguments.

In order to use our data pipeline in our algorithm, the first step is to add a reference to it in our algorithm's initialize function. This is done using the attach_pipeline method, which requires two inputs: a reference to our Pipeline object (which we construct using make_pipeline), and a String name to identify it. 

In [ ]:
# Import Algorithm API functions
from quantopian.algorithm import attach_pipeline


def initialize(context):
    # Attach pipeline to algorithm
    attach_pipeline(
        make_pipeline(),
        'data_pipe'
    )

    # Schedule rebalance function
    schedule_function(
        rebalance,
        date_rule=date_rules.week_start(),
        time_rule=time_rules.market_open()
    )


def before_trading_start(context, data):
    pass


def rebalance(context, data):
    pass

As mentioned above, our pipeline will process data streams and generate an output before the market opens each day. We can get our pipeline's output in before_trading_start using the pipeline_output function, which takes the pipeline name we specified in initialize, and returns the pandas DataFrame generated by our pipeline. For now we can use our rebalance function to log the top 10 rows from our pipeline's output. 

In [ ]:
# Import Algorithm API functions
from quantopian.algorithm import (
    attach_pipeline,
    pipeline_output,
)


def initialize(context):
    # Attach pipeline to algorithm
    attach_pipeline(
        make_pipeline(),
        'data_pipe'
    )

    # Schedule rebalance function
    schedule_function(
        rebalance,
        date_rule=date_rules.week_start(),
        time_rule=time_rules.market_open()
    )

# this is new!!!
def before_trading_start(context, data):
    # Get pipeline output and
    # store it in context
    context.output = pipeline_output(
        'data_pipe'
    )

# this is new!!!
def rebalance(context, data):
    # Display first 10 rows
    # of pipeline output
    log.info(context.output.head(10))

Now, let's add the make_pipeline function we built in Research to our algorithm. Instead of limiting the number of assets like we did for our analysis, our algorithm should consider all assets in the trading universe for which it has a sentiment score. For this we can use the notnull method of our sentiment_score output to create a filter, and get its intersection with the tradable universe using the & operator: 

In [ ]:
# Import Algorithm API functions
from quantopian.algorithm import (
    attach_pipeline,
    pipeline_output,
)

# Pipeline imports
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.psychsignal import stocktwits #this is like a sentiment thing??
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS


def initialize(context):
    # Attach pipeline to algorithm
    attach_pipeline(
        make_pipeline(),
        'data_pipe'
    )

    # Schedule rebalance function
    schedule_function(
        rebalance,
        date_rule=date_rules.week_start(),
        time_rule=time_rules.market_open()
    )


def before_trading_start(context, data):
    # Get pipeline output and
    # store it in context
    context.output = pipeline_output('data_pipe')


def rebalance(context, data):
    # Display first 10 rows
    # of pipeline output
    log.info(context.output.head(10))

# this is new!!!
# Pipeline definition
def make_pipeline():

    base_universe = QTradableStocksUS()

    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )

    return Pipeline(
        columns={
            'sentiment_score': sentiment_score,
        },
        screen=(
            base_universe
            & sentiment_score.notnull()
        )
    )

Our algorithm now selects a tradable universe of assets each day, and produces sentiment data we can use to determine asset allocation within our portfolio. In the next lesson we will learn how to construct an optimal portfolio based on the sentiment scores generated by our data pipeline.